In [8]:
import os
import operator
import json

In [9]:
origin_data_path = './sb-process-1/'
save_data_path = './sb-process-2/'
filenames = ['train', 'test', 'valid']
in_file_name = 'modification.json'
# in_file_names = [i + '.txt' for i in filenames]
# out_file_names = [x + '.json' for x in filenames]

In [7]:
def make_triple(e1, r1, e2):
    res = []
    res.append(e1)
    res.append(r1)
    res.append(e2)
    return res

In [37]:
def make_data(line):
    relation = line.split(':')[0]
#     print(line)
    
    line = line.split(':')[1]
#     stage_txt = filename + ' stage'
    
#     stage = 'STAGE_1'
#     line = stage_txt + ', ' + line
    triples = []
    line_heads = []
    line_relations = []
    line_tails = []
    
    data = {}
    data['triples'] = []
    data['target'] = line
    data['target_txt'] = line
    data['ner2ent'] = {}
    
    target = line
    head_count = 0
    tail_count = 0
    
    
    
#     data['ner2ent'][stage] = line_heads[0]
#     target = target.replace(stage_txt, stage)
#     for r in relations:
#         if target.find(r) != -1:
#             line_relations.append(r)


    for h in heads:
        if target.find(h + ' ') != -1:
            head_count += 1
            line_heads.append(h)
            for k, v in catagories_heads.items():
                if h in v:
                    data['ner2ent'][k + str(head_count)] = h
                    target = target.replace(h + 'es ', k + str(head_count) + ' ')
                    target = target.replace(h + 's ', k + str(head_count) + ' ')
                    target = target.replace(h + ' ', k + str(head_count) + ' ')
    
    for t in tails:
        if target.find(t + ' ') != -1:
            tail_count += 1
            line_tails.append(t)
            for k, v in catagories_tails.items():
                if t in v:
                    data['ner2ent'][k + str(tail_count)] = t
                    target = target.replace(t + 'es ', k + str(tail_count)+ ' ')
                    target = target.replace(t + 's ', k + str(tail_count)+ ' ')
                    target = target.replace(t + ' ', k + str(tail_count)+ ' ')
                    
            
    if len(line_relations) == 1:
        for t in line_tails:
            triples.append(make_triple(line_heads[0], relation, t))
    elif len(line_tails) == 1:
        for h in line_heads:
            triples.append(make_triple(h, relation, line_tails[0]))
    elif len(line_relations) == len(line_tails):
        for h, t in zip(line_heads, line_tails):
            triples.append(make_triple(h, relation, t))
    else:
        print(line_relations)
        print(line_tails)
        print(line)
        
    data['triples'] = triples
    
#     for t in triples:
#         if t[0] not in data['ner2ent'].values():
#             head_count += 1
#             mask = 'STAGE_' + str(head_count)
#             data['ner2ent'][mask] = t[0]
#             target.replace(t[0], mask)
            
#         if t[2] not in data['ner2ent'].values():
#             tail_count += 1
#             for k, v in catagories.items():
#                 if t[2] in v:
#                     data['ner2ent'][k + str(tail_count)] = t[2]
#                     target.replace(t[2], k + str(tail_count))
    
    data['target'] = target
    
    return data

In [38]:
def process(dir_name, save_dir_name, in_file_name, out_file_name):
    data = []
    file_name = in_file_name.split('.')[0]
    with open(dir_name + in_file_name, 'r') as f1:
        for l in f1.readlines():
            l = l[:-1]
            d = make_data(l)
#             d = make_data(l, file_name)
            if d is not None:
#                 print(d['triples'])
#                 print(len(d['triples']))
                if len(d['triples']) == 0:
                    pass
#                     print(l)
#                     print(d)
                else:
                    data.append(d)
            
    with open(save_dir_name + out_file_name, 'w') as f2:
        json.dump(data, f2, indent = 4)
    

In [23]:
relations = ['modification', 'general', 'closeup', 'test']

In [24]:
#     'safety', 'tag', 're-identify',
heads = [
    'open', 'remove', 'put', 'install', 'renew', 'deburr', 're-identify',
    'torque', 'clean', 'loosen', 'shorten', 'apply', 'de-energize',  'retain',
    'get', 'touch-up', 'cut', 'attach', 'strip off',
    'paint', 'seal', 'drill', 'mask', 'ground'
    'affix', 'permit', 'identify',
]

In [43]:
catagories_heads = {}
catagories_heads['MODIFY_'] = [
     'remove', 'install', 'renew', 'deburr', 'torque', 'clean', 
    'loosen', 'shorten', 'apply', 'de-energize',  'retain',
     'touch-up', 'cut',  'paint', 'seal', 'drill', 
]
catagories_heads['MOVE_'] = [
    'ground', 'open', 'put', 'strip off', 'mask', 'attach', 'get',
    
]
catagories_heads['COMMIT_'] = [
    'permit', 'identify', 're-identify',
]

heads = []
for v in catagories_heads.values():
    for t in v:
        heads.append(t)

In [41]:
catagories_tails = {}
catagories_tails['DOOR_'] = [
    'cargo compartment door', 'access door', 'avionics compartment door'
]
catagories_tails['AIRCRAFT_'] = [
    'aircraft', 'aircraft electrical network', 'aircraft electrical circuit',
]
catagories_tails['FACILITY_'] = [
    'access platform',  'clamp', 'duct','oxygen mask', 'surface', 'threaded component',
     'work area', 'warning notice', 'circuit breaker', 'removal', 'installation', 
    'protective finish', 'galley unit', 'dome light', 'electrical actuator', 'retained actuator'
    'electrical bonding', 'carpet', 'lavatory', 'galley unit',
    'fwd cargo compartment', 'bonding connection', 
    'blanking cap', 'electrical connector', 'cable fastening'
    'access', 'fairing', 'pipe', 'fastener', 'hole', 'rivet', 'passenger seat',
    'sidewall panel', 'ceiling panel', 'access panel', 'floor panel', 'valve'
    'item', 'restrictor', 'box', 'washer', 'screw'
]
catagories_tails['MATERIAL_'] = ['material']

tails = []
for v in catagories_tails.values():
    for t in v:
        tails.append(t)


In [46]:
# process(origin_data_path, save_data_path, file_name)
for f in filenames:
    in_file_name = f + '.txt'
    out_file_name = f + '.json'
    process(origin_data_path, save_data_path, in_file_name, out_file_name)

[]
['protective finish', 'material']
touch-up protective finish (refer to amm 51-23-11) with material no. 13-002, 16-002, 16-006 (refer to amm 20-31-00).
[]
['surface', 'material']
apply protective treatment on blank surface material no. 16-001 and mat . no. 16-002 (refer to amm 20-31-00).
[]
['avionics compartment door', 'access platform']
put the correct access platform at the avionics compartment door 822 at the rh side.
[]
['protective finish', 'material']
to renew the protective finish (refer to amm 51-23-11) with material no. 16-002, 16-006 (refer to cml).
[]
['work area', 'material']
to clean the work area with material no. 11-003 (refer to cml),
[]
['protective finish', 'material']
to renew the protective finish with material no. 16-020, material no. 16-001, material no. 16-002 (the internal area), material no. 16-018 (the external area) (refer to cml) .
[]
['protective finish', 'material']
to renew protective finish (refer to amm 51-23-11) with material no. 16-002, 16-006 (ref

In [269]:
s = 'asdsad.xzd'
s.split('.')[0]

'asdsad'

In [262]:
raise(RuntimeError("ee"))

RuntimeError: ee

In [21]:
def make_data(line, filename):
    stage_txt = filename + ' stage'
    
    stage = 'STAGE_1'
#     line = stage_txt + ', ' + line
    triples = []
    line_heads = [stage_txt]
    line_relations = []
    line_tails = []
    
    data = {}
    data['triples'] = []
    data['target'] = line
    data['target_txt'] = line
    data['ner2ent'] = {}
    
    data['target_txt'] = line
    target = line
    head_count = 0
    tail_count = 0
    
    
    
    data['ner2ent'][stage] = line_heads[0]
    target = target.replace(stage_txt, stage)
    for r in relations:
        if target.find(r) != -1:
            line_relations.append(r)
    for t in tails:
        if target.find(t) != -1:
            tail_count += 1
            line_tails.append(t)
            for k, v in catagories.items():
                if t in v:
                    data['ner2ent'][k + str(tail_count)] = t
                    target = target.replace(t + 'es', k + str(tail_count))
                    target = target.replace(t + 's', k + str(tail_count))
                    target = target.replace(t, k + str(tail_count))
                    
            
    if len(line_relations) == 1:
        for t in line_tails:
            triples.append(make_triple(line_heads[0], line_relations[0], t))
    elif len(line_tails) == 1:
        for r in line_relations:
            triples.append(make_triple(line_heads[0], r, line_tails[0]))
    elif len(line_relations) == len(line_tails):
        for r, t in zip(line_relations, line_tails):
            triples.append(make_triple(line_heads[0], r, t))
    else:
        print(line_relations)
        print(line_tails)
        print(line)
        
    data['triples'] = triples
    
#     for t in triples:
#         if t[0] not in data['ner2ent'].values():
#             head_count += 1
#             mask = 'STAGE_' + str(head_count)
#             data['ner2ent'][mask] = t[0]
#             target.replace(t[0], mask)
            
#         if t[2] not in data['ner2ent'].values():
#             tail_count += 1
#             for k, v in catagories.items():
#                 if t[2] in v:
#                     data['ner2ent'][k + str(tail_count)] = t[2]
#                     target.replace(t[2], k + str(tail_count))
    
    data['target'] = target
    
    return data